In [1]:
import numpy as np
import pickle 
import os

import simfuncs as sf

import pygsti
from pygsti.processors.processorspec import QubitProcessorSpec as QPS
from pygsti.algorithms.randomcircuit import create_mirror_rb_circuit
from pygsti.processors import CliffordCompilationRules as CCR
from pygsti.algorithms.randomcircuit import create_random_circuit

/opt/anaconda3/envs/QPL/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Circuit generation

This notebook was used to generate the base circuits in each of the 10 datasets used in the simulation section of the paper. 

We begin by setting the experiment number. **You will want to change this number if you want to avoid writing over an existing dataset**

In [53]:
experiment_number = 4

FILL IN

In [54]:
dataset_size = 5000
num_qubits = 4

path = f'./experiment_{experiment_number}'
try:
    os.makedirs(path)
    os.makedirs(path + '/circuits/', exist_ok = True)
except: 
    print(f'You already have an experiment number {experiment_number}!')

Now we construct (or load) the processor for which the circuits are designed to be run on.

In [55]:
if experiment_number == 0:
    gate_names = ['Gxpi2', 'Gypi2', 'Gxmpi2', 'Gympi2', 'Gxpi', 'Gypi', 'Gzpi', 'Gcnot']


    ring_pspec = QPS(num_qubits = num_qubits, qubit_labels=[i for i in range(num_qubits)],
                                                                gate_names=gate_names,
                                                                availability= {'Gcnot': [(0,1), (1,0),(0,3), (3,0),(1,2), (2,1),(2,3), (3,2)]})
    with open(path + '/pspec.pkl', 'wb') as f:
        pickle.dump(ring_pspec, f)
else:
    with open('./experiment_0/pspec.pkl', 'rb') as f:
        ring_pspec = pickle.load(f)

In [56]:
compilations = CCR.create_standard(ring_pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0)

In the next two cells, we generate a big list of random *i.i.d.*-layered circuits!

In [57]:
max_total_error_per_layer = {1: 0.005, 2:0.02, 3:0.03, 4:0.04}
max_depth = {}
min_f = 0.9
for i, e in max_total_error_per_layer.items():
    max_depth[i] = int(min_f // e) 

print(max_depth)

def sample_width():
    return np.random.randint(1, 5)
    #return 3
def sample_depth(w):
    return np.random.randint(1, max_depth[w] + 1)

def sample_two_q_gate_density():
    # For 3 qubits can't have higher density that 2/3
    return (2/3)*np.random.rand()

# Hardcodes the number of qubits and the connectivity
def sample_qubits(w): 
    if w == 2:
        q1 = np.random.choice([0,1,2,3], 1, replace=False)[0]
        q2 = (q1 + np.random.choice([1, -1], 1)[0]) % 4
        qs = [q1, q2]
        qs.sort()
        return qs
    else:
        qs = list(np.random.choice([0,1,2,3], w, replace=False))
        qs.sort()
        return qs

{1: 180, 2: 45, 3: 30, 4: 22}


In [58]:
ws = []
ds = []
qubitslist = []
two_q_densities = []
circuits = []
idealouts = []
bitstrings = []
circuits_and_idealouts = []
aqs = []

for i in range(dataset_size):
    if i % 100 == 0:
        print(i, end=',')
    w = sample_width()
    d = sample_depth(w)
    xi = sample_two_q_gate_density()
    qubits = tuple(sample_qubits(w))
    ws.append(w)
    ds.append(d)
    qubitslist.append(qubits)
    two_q_densities.append(xi)
    
    c = create_random_circuit(ring_pspec, d,  qubit_labels=qubits, sampler='edgegrab', samplerargs=[xi,])
    active_qubits = [q for q in c.line_labels]
    c = pygsti.circuits.Circuit(c.str.split('@')[0] + '@(0,1,2,3)')
    circuits.append(sf.order_circuit(c))
file_name = 'iid_layer_random_clifford_circuits_v1_instance_0'
pygsti.io.write_circuit_list(path + '/circuits/' + file_name + '.txt', circuits)
with open(path + '/circuits/' + file_name + '_params.pkl', 'wb') as f:
    sample_params = {}
    pickle.dump({'widths':ws, 'depths':ds, 'two_q_densities':two_q_densities, 'qubits':qubitslist}, f)

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,

# Create some mirror circuits

In [59]:
path = f'./experiment_{experiment_number}'

In [60]:
circ_path = path + '/circuits/'
old_circuits = pygsti.io.read_circuit_list(circ_path + '/iid_layer_random_clifford_circuits_v1_instance_0.txt')

max_c_depth = int(max([c.depth for c in old_circuits]) // 6)
max_c_depth

30

In [61]:
max_m_depth = {1: max_c_depth, 2: max_c_depth, 3: max_c_depth, 4: max_c_depth }

def sample_width():
    return np.random.randint(1, 5)
    #return 3
def sample_depth(w):
    return np.random.randint(1, max_m_depth[w] + 1)

def sample_two_q_gate_density():
    # For 3 qubits can't have higher density that 2/3
    return (2/3)*np.random.rand()

# Hardcodes the number of qubits and the connectivity
def sample_qubits(w): 
    if w == 2:
        q1 = np.random.choice([0,1,2,3], 1, replace=False)[0]
        q2 = (q1 + np.random.choice([1, -1], 1)[0]) % 4
        qs = [q1, q2]
        qs.sort()
        return qs
    else:
        qs = list(np.random.choice([0,1,2,3], w, replace=False))
        qs.sort()
        return qs

In [62]:
# %%time
dataset_size = 750


ws = []
ds = []
qubitslist = []
two_q_densities = []
circuits = []
idealouts = []
bitstrings = []
circuits_and_idealouts = []
aqs = []

for i in range(dataset_size):
    if i % 100 == 0:
        print(i, end=',')
    w = sample_width()
    d = sample_depth(w)
    xi = sample_two_q_gate_density()
    qubits = tuple(sample_qubits(w))
    ws.append(w)
    ds.append(d)
    qubitslist.append(qubits)
    two_q_densities.append(xi)

    c, io = create_mirror_rb_circuit(ring_pspec, compilations, 2*d,  qubit_labels=qubits, sampler='edgegrab', samplerargs=[xi,])
    active_qubits = [q for q in c.line_labels]
    c = pygsti.circuits.Circuit(c.str.split('@')[0] + '@(0,1,2,3)')
    circuits.append(sf.order_circuit(c))

file_name = 'mirrored_random_clifford_circuits_v1_instance_0'

pygsti.io.write_circuit_list(path + '/circuits/' + file_name + '.txt', circuits)
with open(path + '/circuits/' + file_name + '_params.pkl', 'wb') as f:
    sample_params = {}
    pickle.dump({'widths':ws, 'depths':ds, 'two_q_densities':two_q_densities, 'qubits':qubitslist}, f)

0,100,200,300,400,500,600,700,

In [63]:
max_mirror_depth = max([c.depth for c in circuits])
assert(max_mirror_depth <= 6*max_c_depth), 'You have some mirror circuits that are too long! Be sure to remove them later!'

In [64]:
print('Done')

Done
